# Lab 4 - Reinforcement Learning

## Read details of the environment & model from a text file
Read: 

- Width, Height
- Noise: probability of not going in the intended direction (then divided by two for two unintentional neighbor directions)
- Immediate rewards of non-goal states
- Terminal (goal) states: their locations and rewards
- Internal Walls: locations

Create the transition matrix (T) and the Rewards matrix according to above details.

"grid1.txt" corresponds to the example in AIMA and the lecture slides. You can double-check various results there.

Python style comments in grid1.txt are just comments, can be ignored/deleted.

Task1 - We ask you to create two more grids (files) of moderate size and with at least one internal wall. And perform the analysis described below on these three grids.

In [ ]:
import numpy as np
file1 = open('grid3.txt', 'r') 
(W, H) = [t(s) for t,s in zip((int,int),file1.readline().split())]
noise = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
immediate_rewards = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
N = W*H #total number of states
NA = 4 #number of actions
T = np.zeros((N,N,NA)) #state transition probabilities
Directions = np.array(['L','R','U','D'])
Rewards = np.zeros((N,1)) + immediate_rewards #immediate rewards
Terminals = np.zeros((N,1)) #terminal states
Walls = np.zeros((N,1)) #(internal) wall states
prob1 = 1-noise
prob2 = noise/2
while True: 
    line = file1.readline() 
    if not line: 
        break
    if line.find('T') != -1:
        (_,x, y, r) = [t(s) for t,s in zip((str,int,int,float),line.split())]
        Terminals[y*W+x] = 1
        Rewards[y*W+x] = r
    if line.find('W') != -1:
        (_,x, y) = [t(s) for t,s in zip((str,int,int),line.split())]
        Walls[y*W+x] = 1     
file1.close()
for i in range(W):
    for j in range(H):
        if i+1<W and Walls[j*W+i+1]==0: #not wall on the right
            T[j*W+i,j*W+i+1,1] += prob1 
            T[j*W+i,j*W+i+1,2] += prob2
            T[j*W+i,j*W+i+1,3] += prob2
        else:
            T[j*W+i,j*W+i,1] += prob1 
            T[j*W+i,j*W+i,2] += prob2
            T[j*W+i,j*W+i,3] += prob2
        if i-1>=0 and Walls[j*W+i-1]==0: #not wall on the left
            T[j*W+i,j*W+i-1,0] += prob1 
            T[j*W+i,j*W+i-1,2] += prob2
            T[j*W+i,j*W+i-1,3] += prob2
        else:
            T[j*W+i,j*W+i,0] += prob1 
            T[j*W+i,j*W+i,2] += prob2
            T[j*W+i,j*W+i,3] += prob2
        if j+1<H and Walls[(j+1)*W+i]==0: #not wall below
            T[j*W+i,(j+1)*W+i,3] += prob1 
            T[j*W+i,(j+1)*W+i,0] += prob2
            T[j*W+i,(j+1)*W+i,1] += prob2
        else:
            T[j*W+i,j*W+i,3] += prob1 
            T[j*W+i,j*W+i,0] += prob2
            T[j*W+i,j*W+i,1] += prob2       
        if j-1>=0 and Walls[(j-1)*W+i]==0: #not wall above
            T[j*W+i,(j-1)*W+i,2] += prob1 
            T[j*W+i,(j-1)*W+i,0] += prob2
            T[j*W+i,(j-1)*W+i,1] += prob2
        else:
            T[j*W+i,j*W+i,2] += prob1 
            T[j*W+i,j*W+i,0] += prob2
            T[j*W+i,j*W+i,1] += prob2    

print(Walls)

## Task 2 - Implement functions

### Value and Policy iteration
Value iteration is provided, implement policy iteration

### TD-Learning and Q-Learning
Q-Learning is implemented, implement TD-Learning

In [ ]:
def value_iteration(gamma, T, Rewards, Terminals, Walls):   
    N, NA = T.shape[0], T.shape[2]
    V = np.zeros((N,1))
    Policy = np.zeros((N,1))
    while True:
        V_old = np.copy(V)
        for s in range(N):
            if Walls[s]==1: continue 
            if Terminals[s]==1:
                V[s] = Rewards[s]
                continue
            Q = np.zeros((NA,1))
            for a in range(NA):
                Q[a] = Rewards[s] + gamma*np.dot(T[s,:,a],V)
            V[s] = np.max(Q)
            Policy[s] = np.argmax(Q)
        if np.sum(np.abs(V-V_old))<1e-10:
            break
    return V, Policy


def policy_iteration(gamma, T, Rewards, Terminals, Walls): 
    N, NA = T.shape[0], T.shape[2]
    V = np.zeros((N,1))
    Policy = np.zeros((N,1))    
    # Implement PI
    unchanged = True
    
    while True:
        while True:
            delta = 0
            PI_old = np.copy(Policy)
            #Policy Evaluation
            for s in range(N):
                v = V[s]
                if Walls[s] == 1:
                    continue
                if Terminals[s] == 1:
                    V[s] = Rewards[s]
                    continue

                V[s] = Rewards[s] + gamma*np.dot(T[s,:,int(Policy[s])],V)
                delta = max(delta, np.abs(v-V[s]))
            if delta < 1e-10:
                break
                
        unchanged = True
        
        #Policy Improvement
        for s in range(N):
            p_old = Policy[s]
            Q = np.zeros((NA, 1))
            for a in range(NA):
                Q[a] = Rewards[s] + gamma*np.dot(T[s,:,a],V)
                
            #print(Q)
            if np.max(Q) > Q[int(Policy[s])]:
                Policy[s] = np.argmax(Q)
                unchanged = False
            
        if unchanged:
            return V, Policy
 
         
    

def TD_learning(N_episodes, alpha, gamma, T, Rewards, Terminals, Walls, Policy): 
    N, NA = T.shape[0], T.shape[2]
    V = np.zeros((N,1))
    N_s = np.zeros((N, 1))
    sp, ap, rp = None, None, None
    # Implement TD
    for e in range(N_episodes):
        s = 0
        sp, ap, rp = None, None, None
        r = Rewards[s]
        while True:
            if N_s[s] == 0:
                V[s] = r
            if not sp == None:
                N_s[sp] += 1
                #print(V[sp], (alpha/N_s[sp]) * (r + gamma * V[s] - V[sp]))
                V[sp] = V[sp] + (alpha/N_s[sp]) * (r + gamma * V[s] - V[sp]) 
            if Terminals[s] == 1:
                break
            else:
                sp = s
                ap = Policy[s]
                rp = r
            #
            s = np.random.choice(range(N), p = T[sp,:,int(ap)])
            r = Rewards[s]
            #print(T[sp,:,int(ap)])
            
    
    
    #
    return V, Policy

def Q_learning(N_episodes, epsilon, alpha, gamma, T, Rewards, Terminals, Walls): 
    N, NA = T.shape[0], T.shape[2]
    Q = np.zeros((N,NA))
    for e in range(N_episodes):
        s = int(np.floor(np.random.uniform(0,N-1)))
        while Terminals[s]==1 or Walls[s]==1:
            s = int(np.floor(np.random.uniform(0,N-1)))
        while Terminals[s]==0:
            u = np.random.uniform(0,1)
            if u<epsilon:
                a = int(np.floor(np.random.uniform(0,NA)))
            else:
                a = np.argmax(Q[s,:])
            u = np.random.uniform(0,1)
            s1 = np.argmax(u<np.cumsum(T[s,:,a]))
            Q[s,a] += alpha*(Rewards[s1] + gamma*np.max(Q[s1,:]) - Q[s,a])
            s = s1
        epsilon = epsilon*0.9999 # Modify here for other annealing regimes
        alpha = alpha*0.9999 # Modify here for other annealing regimes
    V = np.max(Q,axis=1)
    V = V[:,None]
    V[Terminals==1] = Rewards[Terminals==1] 
    Policy = np.array(np.argmax(Q,axis=1)).reshape((N,1))
    return V, Policy

V1, Policy1 = value_iteration(1, T, Rewards, Terminals, Walls)
print(V1)
print(Policy1)
V2, Policy2 = policy_iteration(1, T, Rewards, Terminals, Walls)
print('Policy Iteration \n', V2)
print('Policy Iteration \n', Policy2)
V3, Policy3 = TD_learning(1000, 0.9, 1, T, Rewards, Terminals, Walls, Policy1)
print('TD_learning \n', V3)
print('TD_learning \n', Policy3)
V4, Policy4 = Q_learning(1000, 0.9, 1, 1, T, Rewards, Terminals, Walls)
print('Q_learing \n', V4)
print('Q_learning \n', Policy4)

## Task 3 - Experiments

For the three grids you have, perform the following analysis and include them in your report,.

- Compare value and policy iterations in terms of convergence time. Relate it to computational complexity, current implementation and number of iterations needed.

- How are optimal policies change with immediate reward values? Show some examples (similar to Figure 17.2b in AIMA)

- Compare TD-Learning and Q-Learning results with each other. Also with value and policy iterations. Remember that value and policy iteration solve Markov Decision Processes where we know the model (T and Rewards). TD-Learning and Q-Learning are (passive and active, respectively) Reinforcement Learning methods that don't have the model but use data from simulations. Data are simulated from the model we know, but the model is not used n TD or Q-Learning.

- What are the effects of epsilon and alpha values and how they are modified?

- What is the effect of number of episodes?

In [ ]:
import time

gamma = 1
alpha = 0.9
epsilon = 0.5
N_episodes = 20000

#Value iteration
VI_time = time.perf_counter_ns()
V_VI, Policy_VI = value_iteration(gamma, T, Rewards, Terminals, Walls)
VI_time = time.perf_counter_ns() - VI_time
#Policy iteration
PI_time = time.perf_counter_ns()
V_PI, Policy_PI = policy_iteration(gamma, T, Rewards, Terminals, Walls)
PI_time = time.perf_counter_ns() - PI_time
#TD-Learning
V_TD, Policy_TD = TD_learning(N_episodes, alpha, gamma, T, Rewards, Terminals, Walls, Policy_VI)
#Q-Learning
V_Q, Policy_Q = Q_learning(N_episodes, epsilon, alpha, gamma, T, Rewards, Terminals, Walls)

print(VI_time, 'Time value iteration')
print(PI_time, 'Time policy iteration')

print(V_VI.reshape((H,W)))
maze = Directions[Policy_VI.astype(int)]
maze[Walls==1] = 'W'
maze[Terminals==1] = 'G'
print(maze.reshape((H,W)))
#

print(V_PI.reshape((H,W)))
maze = Directions[Policy_PI.astype(int)]
maze[Walls==1] = 'W'
maze[Terminals==1] = 'G'
print(maze.reshape((H,W)))

print(V_TD.reshape((H,W)))
maze_Q = Directions[Policy_TD.astype(int)]
maze_Q[Walls==1] = 'W'
maze_Q[Terminals==1] = 'G'
print(maze_Q.reshape((H,W)))

print(V_Q.reshape((H,W)))
maze_Q = Directions[Policy_Q.astype(int)]
maze_Q[Walls==1] = 'W'
maze_Q[Terminals==1] = 'G'
print(maze_Q.reshape((H,W)))

## Task 4 (extra credit) - What if diagonal moves were possible?

Repeat the experiments in one of these settings. You don't have to stick to the code provided here. But, you can only use Numpy (and matplotlib).

You should decide on how to distribute the noise associated with actions across (next) states. For example, in the scenario above we have probability of going Up with the Up action is 1-noise. The probability of going Left with the Up action is noise/2. The same for going Right.